# Advent of Code 2022

## Day 8: Treetop Tree House

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

### Imports

In [ ]:
import numpy as np
from typing import Iterator

### File Reading

In [ ]:
def read_array(filename: str) -> np.array:
    rv = []
    with open(filename) as file:
        for line in file:
            rv.append([int(char) for char in line.strip()])
    return np.array(rv, dtype=int)

In [ ]:
INPUT_FILE = 'data/input08.txt'

### Light Beam Generators

The `beam` method returns a list of beams from each direction. Each beam is an iterator of points.

If the grid is 4x3 then there are 14 beams, that is 4 from the top, 4 from the bottom, 3 from the left and 3 from the right.

In [ ]:
def lr_beam(shape: tuple[int, int], *, y: int) -> Iterator[tuple[int, int]]:
    height, width = shape
    for x in range(0, width):
        yield y, x


def rl_beam(shape: tuple[int, int], *, y: int) -> Iterator[tuple[int, int]]:
    height, width = shape
    for x in reversed(range(0, width)):
        yield y, x


def tb_beam(shape: tuple[int, int], *, x: int) -> Iterator[tuple[int, int]]:
    height, width = shape
    for y in range(0, height):
        yield y, x


def bt_beam(shape: tuple[int, int], *, x: int) -> Iterator[tuple[int, int]]:
    height, width = shape
    for y in reversed(range(0, height)):
        yield y, x

def beams(shape: tuple[int, int]) -> list[Iterator[tuple[int, int]]]:
    rv = []
    rv.extend([lr_beam(shape, y=y) for y in range(shape[0])])
    rv.extend([rl_beam(shape, y=y) for y in range(shape[0])])
    rv.extend([tb_beam(shape, x=x) for x in range(shape[1])])
    rv.extend([bt_beam(shape, x=x) for x in range(shape[1])])
    return rv

### Helper Functions

In [ ]:
# helper function to count zeros in a numpy array, uses count_nonzero
def count_zero(arr: np.ndarray):
    return np.count_nonzero(arr == 0)

### Part 1

In [ ]:
# shines beams from all directions and counts the number of hits on each tree
def count_hits(trees: np.ndarray):

    # the number of times a beam hits each tree
    hits = np.zeros(trees.shape, dtype=int)

    for iterator in beams(trees.shape):

        beam_height = -1

        for y, x in iterator:

            tree_height = trees[y,x]

            if tree_height > beam_height:
                hits[y,x] += 1
                beam_height = tree_height

    return hits

In [ ]:
def main():
    trees = read_array(INPUT_FILE)
    hits = count_hits(trees)
    num_hit_trees = np.count_nonzero(hits)
    print(f'The number of visible trees is {num_hit_trees}.')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
# the beams going outwards from a given tree
def outward_beams(shape: tuple[int, int], loc_y: int, loc_x: int) -> list[Iterator[tuple[int, int]]]:
    return [
        (((y, x) for y, x in lr_beam(shape, y=loc_y) if x > loc_x)),
        (((y, x) for y, x in rl_beam(shape, y=loc_y) if x < loc_x)),
        (((y, x) for y, x in tb_beam(shape, x=loc_x) if y > loc_y)),
        (((y, x) for y, x in bt_beam(shape, x=loc_x) if y < loc_y))
    ]

In [ ]:
def calc_scene_score_of_tree_for_direction(trees: np.ndarray, origin_height: int, beam: Iterator[tuple[int, int]]) -> int:

    count = 0

    for y, x in beam:

        count += 1

        # stop at higher or equal tree (or at the edge, which is the end of the iterator)
        if trees[y,x] >= origin_height:
            break

    return count

In [ ]:
def calc_scene_score_of_tree(trees: np.ndarray, origin_y: int, origin_x: int) -> int:
    origin_height = trees[origin_y,origin_x]
    score = 1
    for beam in outward_beams(trees.shape, origin_y, origin_x):
        current_direction_score = calc_scene_score_of_tree_for_direction(trees, origin_height, beam)
        score *= current_direction_score
    return score

In [ ]:
def main():

    trees = read_array(INPUT_FILE)

    maximum = -1
    for origin_y in range(trees.shape[0]):
        for origin_x in range(trees.shape[1]):
            score = calc_scene_score_of_tree(trees, origin_y, origin_x)
            if score > maximum:
                maximum = score

    print(f'The highest score for a tree is {maximum} points.')

In [ ]:
if __name__ == '__main__':
    main()